In [3]:
import streamlit as st
import requests
from woocommerce import API
import zipfile
import os
from PIL import Image
from io import BytesIO
import dotenv

# Load environment variables from .env file
dotenv.load_dotenv()

# Access environment variables
WC_API_URL = os.environ.get('WC_API_URL')
WC_CONSUMER_KEY = os.environ.get('WC_CONSUMER_KEY')
WC_CONSUMER_SECRET = os.environ.get('WC_CONSUMER_SECRET')

In [4]:
# Initialize the WooCommerce API
wcapi = API(
    url=WC_API_URL,
    consumer_key=WC_CONSUMER_KEY,
    consumer_secret=WC_CONSUMER_SECRET,
    version="wc/v3"
)

In [5]:
def fetch_product_images(product_id):
    """Fetch product images from WooCommerce using the product ID."""
    response = wcapi.get(f"products/{product_id}")
    if response.status_code == 200:
        product_data = response.json()
        images = product_data.get('images', [])
        image_urls = [img['src'] for img in images]
        return image_urls
    else:
        st.error("Failed to fetch product data. Please check the product ID.")
        return []

def download_images(image_urls):
    """Download images from the provided URLs."""
    images = []
    for url in image_urls:
        response = requests.get(url)
        if response.status_code == 200:
            img = Image.open(BytesIO(response.content))
            images.append(img)
        else:
            st.warning(f"Failed to download image from {url}")
    return images

def create_zip(images, zip_filename="product_images.zip"):
    """Create a zip file from the downloaded images."""
    with zipfile.ZipFile(zip_filename, 'w') as zipf:
        for i, img in enumerate(images):
            img_filename = f"image_{i+1}.png"
            img.save(img_filename)
            zipf.write(img_filename)
            os.remove(img_filename)  # Clean up the individual image file
    return zip_filename

def main():
    st.title("WooCommerce Product Image Downloader")

    # Input for product ID
    product_id = st.text_input("Enter Product ID:")

    if product_id:
        # Fetch image URLs
        image_urls = fetch_product_images(product_id)
        
        if image_urls:
            st.success(f"Found {len(image_urls)} images for product ID {product_id}.")
            
            # Download images
            images = download_images(image_urls)
            
            if images:
                # Create a zip file
                zip_filename = create_zip(images)
                
                # Provide download button
                with open(zip_filename, "rb") as f:
                    st.download_button(
                        label="Download Images as Zip",
                        data=f,
                        file_name=zip_filename,
                        mime="application/zip"
                    )
                
                # Clean up the zip file
                os.remove(zip_filename)
            else:
                st.warning("No images were downloaded.")
        else:
            st.warning("No images found for the given product ID.")


2025-02-02 14:44:41.822 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:44:42.164 
  command:

    streamlit run c:\Users\yeasi\Projects\Streamlit\.conda\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-02-02 14:44:42.165 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:44:42.166 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:44:42.168 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:44:42.170 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:44:42.172 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-02 14:44:42.174 Session

In [ ]:
%streamlit run woocommerce_app.py